<a href="https://colab.research.google.com/github/whart222/pao/blob/master/doc/besancon27.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **A PAO Example: Solving Besancon27**

NOTE: This notebook assumes that PAO and related solvers are installed in the python environment that is used to launch jupyter.

## Creating and Solving a PAO Model using Pyomo

In [1]:
import pyomo.environ as pe
from pao.bilevel import *

In [2]:
# Example 2.7 from
#
# Near-Optimal Robust Bilevel Optimization
#   M. Besancon, M. F. Anjos and L. Brotcorne
#   arXiv:1908.04040v5 (2019)
#
# Optimal solution: (x,v) = (0,1)

M = pe.ConcreteModel()

M.x = pe.Var(bounds=(0.5,None),initialize=1)  # Lower bound set to 0.5
M.v = pe.Var(initialize=1)

M.o = pe.Objective(expr=M.x, sense=pe.minimize)
M.c = pe.Constraint(expr= M.v >= 1 - M.x/10)

M.L = SubModel(fixed=M.x)
M.L.o = pe.Objective(expr=M.v, sense=pe.maximize)
M.L.c = pe.Constraint(expr= 1 + M.x/10 >= M.v)

In [3]:
# Available Solvers
print(list(Solver))

['pao.lbp.FA', 'pao.lbp.PCCG', 'pao.lbp.REG', 'pao.lbp.interdiction', 'pao.pyomo.FA', 'pao.pyomo.PCCG', 'pao.pyomo.REG']


In [4]:
solver = Solver("pao.pyomo.FA")
results = solver.solve(M, solver="glpk")

In [5]:
# The results object contains information about the problem and solver
print(results)

Problem:
- lower_bound: 0.5
  name: 'unknown'
  number_of_binary_variables: None
  number_of_constraints: None
  number_of_continuous_variables: None
  number_of_integer_variables: None
  number_of_objectives: None
  number_of_variables: None
  sense: 'minimize'
  upper_bound: 0.5
Solver:
- best_feasible_objective: 0.5
  config: <pyomo.common.config.ConfigDict object at 0x7fba63a99470>
  lmp_solver: 'pao.lbp.FA'
  name: 'pao.pyomo.FA'
  rc: None
  solver_time: None
  termination_condition: <TerminationCondition.optimal: 'optimal'>
  wallclock_time: 0.4276437759399414


In [6]:
# The solution is loaded back into the model
M.pprint()

1 Set Declarations
    zzz_PAO_SlackVariables_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    0 :      {}

3 Var Declarations
    v : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :  None : 0.94999999999709 :  None : False : False :  Reals
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :   0.5 :   0.5 :  None : False : False :  Reals
    zzz_PAO_SlackVariables : Size=0, Index=zzz_PAO_SlackVariables_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain

1 Objective Declarations
    o : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize :          x

1 Constraint Declarations
    c : Size=1, Index=None, Active=True
        Key  : Lower : Body          : Upper : Active
        None :  -Inf : 1 - 0.1*x - v :   0.0 :   True

1 S